In [1]:
import os
import pandas as pd
import requests

In [2]:
folder='all'
columns=['handle','url','provider-homepage','template-type']
all_items=[]
for file in os.listdir(folder):
    if file.endswith('.rdf'):
        with open(folder+'/'+file,'r') as rdf_file:
            rdf_string=rdf_file.read()
            rdf_list= rdf_string.split('\n')
            rdf_item_dict={}
            for rdf_item in rdf_list:
                if ':' not in rdf_item: 
                    continue
                key=rdf_item.split(':')[0]
                if 1==2 or key.lower() in columns:
                    value=rdf_item.replace(key+':','')
                    rdf_item_dict[key.strip().lower()]=value.strip()
            all_items.append(rdf_item_dict)

In [ ]:
all_items_status=[]
for item in all_items[0:20]:
    if 'template-type' in item:
        if item['template-type']=='ReDIF-Archive 1.0' and item['url'][:4]=='http':
            url=item['url']
            try: 
                r = requests.get(url)
            except:
                r = 'Refused'
            item['status']=r
            all_items_status.append(item)

In [ ]:
df=pd.DataFrame(all_items_status)
df.shape

In [25]:
df

,handle,status,template-type,url
0,RePEc:aal,refused,ReDIF-Archive 1.0,http://www.druid.dk/RePEc/aal/
1,RePEc:aay,<Response [200]>,ReDIF-Archive 1.0,http://aaye.org/RePEc/aay/
2,RePEc:abn,<Response [200]>,ReDIF-Archive 1.0,http://cla.auburn.edu/econwp/RePEc/abn/
3,RePEc:acb,<Response [200]>,ReDIF-Archive 1.0,https://www.cbe.anu.edu.au/RePEc/acb/index.htm
4,RePEc:adl,refused,ReDIF-Archive 1.0,http://economics.adelaide.edu.au/research/pape...
5,RePEc:adr,<Response [200]>,ReDIF-Archive 1.0,http://annales.ensae.fr/RePEc/adr/
6,RePEc:ads,<Response [404]>,ReDIF-Archive 1.0,http://www.sss.ias.edu/files/RePEc/ads/
